In [79]:
####################################################################################
# This test code shows how new_basin_cities looks like
# basin at each city
distance=100
min_of_cities=0
max_of_basinnum=1000
target_city = 1676
####################################################################################

h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
basin_cities_path = f'{h08dir}/global_city/dat/riv_num_/basin_to_cities_{distance}km.json'

with open(basin_cities_path, 'r') as json_file:
    tmp = json.load(json_file)
    
sorted_keys = sorted(tmp.keys(), key=lambda k: float(k))

target_key_list = []

for key in sorted_keys:
    value = tmp[key]
    if min_of_cities < len(value) and float(key) < max_of_basinnum:
        for i in value:
            if int(float(i)) == target_city:
                target_key_list.append(key)
                
print(f'target_city: {target_city}')
print(f'target_key_list: {target_key_list}')

target_city: 1676
target_key_list: ['4.0']


In [80]:
# This test code shows how cities in target_basin looks like

target_key = '4.0'
value = tmp[target_key]
print(f'target_key: {target_key}')
print(f'value: {value}')

target_key: 4.0
value: ['9', '71', '76', '118', '200', '579', '691', '729', '746', '1333', '1551', '1579', '1667', '1675', '1676', '1747']


In [81]:
# This test code shows riv_num index of tmp dictionary

keys_lst = list(tmp.keys())
key_index = keys_lst.index(target_key)
print(f'rivnum_key_index: {key_index}')

rivnum_key_index: 10


In [82]:
# This test code shows ava & mpcy

import pandas as pd

#mpcy_int_path = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city/dat/cty_lst_/gpw4/mcy_pop_water_12region.txt'
#column_names = ['index', 'flag', 'rate', 'wup', 'pop', 'ava', 'mpcy', 'potential', 'region', 'country', 'cityname', '12region']
#df = pd.read_csv(mpcy_int_path, delimiter='|', header=None, names=column_names)

mpcy_int_path = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08/wsi/dat/availablewater/updown_availability_100km_abandon_revised.txt'
df = pd.read_csv(mpcy_int_path, delimiter='|', header=0)
df_filtered_test = df[df['index'] == target_city]
display(df_filtered_test.head())

,index,flag,rate,wup,pop,ava,mpcy,potential,region,country,cityname,12region,ava_int,mpcy_int
1675,1676,False,1.041073,283.236,294869.21875,2.579747e+07,87.48785,4.0,northern africa,Sudan,Al Gadarif,Northern Africa,2.579747e+07,87.48785


In [83]:
import numpy as np

#######################################################################################
# upstream demand is not considered
# to check upstream_dem => /abandon_prf_or_intake.ipynb/init_dis_dem_tmp(key_index)
#######################################################################################

def discharge_withincity(city_num, riv_num):
    riv_num = int(float(riv_num))
    print(f'city_num: {city_num}, riv_num: {riv_num}')
    distance = 100
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
    rivout_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
    rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
    rivout_gl5 = np.fromfile(rivout_path, dtype='float32').reshape(2160, 4320)
    rivnum_gl5 = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320)
    prf_dir = f'{h08dir}/global_city/dat/cty_prf_'
    int_dir = f'{h08dir}/global_city/dat/cty_int_/{distance}km_samebasin'
    
    prf_path = f'{prf_dir}/vld_cty_/city_{city_num:08}.gl5'
    int_path = f'{int_dir}/city_{city_num:08}.gl5'
    
    prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)
    intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
    
    dis_pprf = np.sum(rivout_gl5[(prf==1) & (rivnum_gl5==riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year
    dis_sprf = np.sum(rivout_gl5[(prf==1) & (rivnum_gl5!=riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year
    dis_pint = np.sum(rivout_gl5[(intake==1) & (rivnum_gl5==riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year
    dis_sint = np.sum(rivout_gl5[(intake==1) & (rivnum_gl5!=riv_num)])* 60 * 60 * 24 * 365 / (1000) # m3/year

    print(f'dis_pprf: {dis_pprf/1e9}')
    print(f'dis_sprf: {dis_sprf/1e9}')
    print(f'dis_pint: {dis_pint/1e9}')
    print(f'dis_sint: {dis_sint/1e9}')

    prf_total = dis_pprf + dis_sprf
    int_total = dis_pint + dis_sint

    print(f'prf_total: {prf_total/1e9}')
    print(f'int_total: {int_total/1e9}')

discharge_withincity(target_city, target_key)

city_num: 1676, riv_num: 4
dis_pprf: 0.001677409420715332
dis_sprf: 0.0
dis_pint: 0.02579747392089844
dis_sint: 0.0
prf_total: 0.001677409420715332
int_total: 0.02579747392089844


In [ ]:
# Hyderabad, India
# to check upstream_dem => /abandon_prf_or_intake.ipynb/init_dis_dem_tmp(key_index)
"""
city_num: 40, riv_num: 53
dis_pprf: 0.0
dis_sprf: 0.299694276421875
dis_pint: 4.47016887
dis_sint: 0.0
prf_total: 0.299694276421875
int_total: 4.47016887
#
memo_dict.keys(): dict_keys([40, 164, 287, 352, 371, 621, 833, 882, 894, 1088, 1119, 1255, 1261, 1497, 1550, 1576, 1639, 1683])
--------------------------------------------------
memo_dict[40]: {'dem_org': 6983691954.091013, 'ava_org': 4470168870.0, 'upstream_dem': 519964019.0, 'ava_mod': 3950204851.0, 'dem_mod': 3950204851.0, 'uup': array([1255, 1261])}
"""